## Imports and common functions

In [118]:
import seaborn as sb
import pandas as pd
import numpy as np
import reverse_geocoder as rg

In [25]:
def load_dat_file(file_name, n_components):  
    """
    Loads in .dat files as lists. 
    
    Inputs: 
        file_name (string): name of file to load in
        n_components (int): number of componenets in row
        
    Outputs:
        (list): list of rows in data file
    """
    new_lines = []
    with open(file_name) as input_file:
        for line in input_file:
            new_line = [x.strip() for x in line.split('|')]
            
            if len(new_line) == n_components:
                new_lines.append(new_line)
    return new_lines

## Load in data

In [16]:
# load in data files
raw_checkin_data = load_dat_file('./raw_data/checkins.dat', 6)
raw_ratings_data = load_dat_file('./raw_data/ratings.dat', 3)
raw_socialgraph_data = load_dat_file('./raw_data/socialgraph.dat', 2)
raw_user_data = load_dat_file('./raw_data/users.dat', 3)
raw_venues_data = load_dat_file('./raw_data/venues.dat', 3)

In [23]:
# convert above data to pandas for easy analysis 
checkin_df = pd.DataFrame(raw_checkin_data[1:], columns=raw_checkin_data[0])
ratings_df = pd.DataFrame(raw_ratings_data[1:], columns=raw_ratings_data[0])
socialgraph_df = pd.DataFrame(raw_socialgraph_data[1:], columns=raw_socialgraph_data[0])
user_df = pd.DataFrame(raw_user_data[1:], columns=raw_user_data[0])
venues_df = pd.DataFrame(raw_venues_data[1:], columns=raw_venues_data[0])

In [24]:
# export data as csv 
checkin_df.to_csv('./raw_data/checkin.csv', index=False)
ratings_df.to_csv('./raw_data/ratings.csv', index=False)
socialgraph_df.to_csv('./raw_data/socialgraph.csv', index=False)
user_df.to_csv('./raw_data/user.csv', index=False)
venues_df.to_csv('./raw_data/venues.csv', index=False)

## Data Cleaning

In [114]:
venues_df = venues_df[(venues_df['latitude'] != '') & (venues_df['longitude'] != '')]
venues_df['latitude'] = pd.to_numeric(venues_df['latitude'])
venues_df['longitude'] = pd.to_numeric(venues_df['longitude'])

In [115]:
# bound box to (roughly) united states only
top_left = (48.313833, -126.350714)
bottom_left = (24.489072, -126.350714)
top_right = (48.313833, -67.288219)
bottom_right = (24.489072, -67.288219)

subset_venues_df = venues_df[(venues_df['latitude'] < top_left[0]) & (venues_df['latitude'] > bottom_right[0]) &
                             (venues_df['longitude'] > top_left[1]) & (venues_df['longitude'] < bottom_right[1])]

In [141]:
# assign cities and countries
current_reverse_geocode = rg.search(zip(subset_venues_df['latitude'], subset_venues_df['longitude']))
cities = [x['name'] for x in current_reverse_geocode]
countries = [x['cc'] for x in current_reverse_geocode]

subset_venues_df['city'] = cities
subset_venues_df['country'] = countries

/Users/kapilgarg/miniconda3/envs/sccs/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kapilgarg/miniconda3/envs/sccs/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [145]:
subset_venues_df['city'].value_counts()

New York City        17617
Chicago              13569
San Francisco        13168
Manhattan            11109
Long Island City     10424
Washington, D.C.      7483
Toronto               7046
Seattle               6591
San Diego             5741
Philadelphia          5358
Brooklyn              4416
Austin                3842
Portland              3833
Minneapolis           3776
Lincolnwood           3701
Boston                3412
Atlanta               3407
Weehawken             3323
Paradise              3057
Glendale              3056
Montreal              2854
Denver                2470
Phoenix               2415
La Jolla              2410
Baltimore             2367
Dallas                2359
Arlington             2343
Paradise Valley       2308
Hoboken               2269
Houston               2242
                     ...  
Rumford                  1
El Dorado Springs        1
Saint-Tite               1
Earlville                1
Gwinn                    1
Cookshire-Eaton          1
R

In [147]:
subset_venues_df['country'].value_counts()

US    1049537
CA      31876
MX       1823
BS        122
BM         30
Name: country, dtype: int64